In [ ]:
# 📦 Step 1: Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity
import random
import tensorflow as tf


In [ ]:
# 📁 Step 2: Set Paths
dataset_path = "dataset"
test_path = "testing_folder"
model_path = "model/rat_cnn_model.keras"
# 📂 Step 3: Create Dataset Directory if it doesn't exis    t

In [ ]:
# 🖼️ Step 3: Data Preprocessing
img_size = (150, 150)
batch_size = 16

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


In [ ]:
# 🧠 Step 4: Build CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# 🚀 Step 5: Train Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)


In [ ]:
# 💾 Step 6: Save Model
os.makedirs("model", exist_ok=True)
model.save(model_path)


In [ ]:
# 📈 Step 7: Plot Accuracy & Loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.show()


In [ ]:
# 🔍 Step 8: Predict Image in testing_folder
model = load_model(model_path)

test_img_name = os.listdir(test_path)[0]
test_img_path = os.path.join(test_path, test_img_name)

img = load_img(test_img_path, target_size=img_size)
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)[0][0]
result = "Rat Detected 🐀" if prediction > 0.5 else "No Rat Detected ❌"

print(f"Prediction: {result} ({prediction:.2f})")

plt.imshow(load_img(test_img_path))
plt.axis('off')
plt.title(result)
plt.show()
